# Experiment Notebook: How do different settings of the FEI Savings Rate process affect User FEI Capital Allocation?

# Table of Contents
* [Experiment Summary](#Experiment-Summary)
* [Experiment Assumptions](#Experiment-Assumptions)
* [Experiment Setup](#Experiment-Setup)
* [Analysis 1: ...](#Analysis-1:-...)

# Experiment Summary 

The purpose of this notebook is to assess the impact that various settings of the FEI Savings Rate process have on User FEI Capital Allocation across Liquidity Pool, Money Market, and FEI Savings Deposits.

# Experiment Assumptions

See [assumptions document](../../ASSUMPTIONS.md) for further details.

# Experiment Setup

We begin with several experiment-notebook-level preparatory setup operations:

* Import relevant dependencies
* Import relevant experiment templates
* Create copies of experiments
* Configure and customize experiments 

Analysis-specific setup operations are handled in their respective notebook sections.

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook configuration methods, such as loading IPython modules
import setup

import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px

import experiments.notebooks.visualizations as visualizations
from experiments.run import run
from experiments.utils import display_code

In [ ]:
from operator import lt, gt

In [ ]:
from scipy.stats import norm

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
import plotly.io as pio
png_renderer = pio.renderers["png"]
png_renderer.width = 1200
png_renderer.height = 500
# png_renderer.scale = 1

pio.renderers.default = "png"

In [ ]:
# Enable/disable logging
logger = logging.getLogger()
logger.disabled = False

In [ ]:
# Import experiment templates
import experiments.default_experiment as default_experiment

In [ ]:
# Create a simulation for each analysis
simulation_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

In [ ]:
def gen_norm_rv(n, mu, sigma):
    return norm.rvs(loc=mu, scale=sigma, size=1, random_state=n)[0]

In [ ]:
cam_deposits = [
    "fei_liquidity_pool_user_deposit",
    "fei_money_market_user_deposit",
    "fei_savings_user_deposit",
    "fei_idle_user_deposit",
]

In [ ]:
parameter_overrides = {
#     "target_rebalancing_condition": [gt, lt], # Simulate decrease and increase of stable PCV
#     "target_stable_pcv_ratio": [0.2, 0.5], # Simulate decrease and increase of stable PCV
#     "rebalancing_period": [int(365 / 4)],
#     "yield_withdrawal_period": [999],  # Disable yield-withdrawal policy
#     "yield_reinvest_period": [999],  # Disable yield-reinvestment policy
    "capital_allocation_fei_deposit_variables": [
            cam_deposits,
    ],
    "capital_allocation_rebalance_duration": [30],
    "fei_savings_rate_process": [
         lambda _run, timestep: 0.01,
         lambda _run, timestep: 0.03,
         lambda _run, timestep: 0.01 if timestep < 365 / 4 else (0.03 if timestep < 365 * 3/4 else 0.01),
    ],
    #"money_market_utilization_rate_process": [
    #    lambda _run, timestep: 0.7, #+ gen_norm_rv(timestep, 0, 0.01),
    #]
}


In [ ]:
# Experiment configuration

# Override default experiment number of Monte Carlo Runs
simulation_1.runs = 5

# Override default experiment System Initial State
simulation_1.model.initial_state.update({})

# Override default experiment System Parameters
simulation_1.model.params.update(parameter_overrides)

## KPI computation (temp)

In [ ]:
def compute_vol(x):
    return x.pct_change().std()

In [ ]:
def compute_max(x):
    return x.max()

In [ ]:
def compute_min(x):
    return x.min()

In [ ]:
def compute_final_val(x):
    return x.iloc[-1]

In [ ]:
def compute_mdd(x):
    window = 30

    rolling_max = x.rolling(window).max()
    dd = x/rolling_max - 1.0
    
    mdd = dd.rolling(window).min()
    return mdd.min()

In [ ]:
fn_list = [compute_vol, compute_max, compute_min, compute_final_val, compute_mdd]
fn_names = ['volatility', 'max', 'min', 'final value', 'max dd']
fn_dict = dict(zip(fn_names, fn_list))

In [ ]:
def generate_emp_distribution_kpi(fn, df, variable, start=None, end=None):
    L = dict()
    for i in list(df['subset'].value_counts().index):
        V = dict()
        for j in list(df['run'].value_counts().index):
            x = df.query('run==@j and subset==@i')[variable].iloc[start:end]
            v = fn(x)
            V[j] = v
            
        L[i] = V
        
    return pd.DataFrame(L)

In [ ]:
def compute_metric_set_for_variable(df, fn_dict, variable, start=None, end=None):
    L = dict()
    for fn_name, fn in fn_dict.items():
        metric_avg = generate_emp_distribution_kpi(fn, df, variable, start=start, end=end).mean(axis=0)
        L[fn_name] = metric_avg
        
    return pd.DataFrame(L)

# Analysis 1: Effect of FEI Savings Rate on User FEI Capital Allocation

In this analysis we look at the effect of setting the FEI savings rate in three ways:
- As a constant process below the money market supply rate
- As a constant process above the money market supply rate
- As a step function which moves between the two processes

We assume the demand for populating each deposit will depend on the rate of yield offered by the deposit, and gauge to what extent the difference in deposit yield affects how deposits are populated. We also look at the effect this has on Liquidity Pools, and a few other downstream effects on mechanism-specific metrics.

In [ ]:
# Analysis-specific setup

In [ ]:
# Experiment execution
df, exceptions = run(simulation_1)

In [ ]:
# Post-processing and visualizations

### visualization of parameter sweep:

we sweep the fei savings rate process for three values, two constant processes and a step function

In [ ]:
df.query('run==1').plot(x='timestamp', y=['fei_savings_rate'], color='subset')

In [ ]:
df.query('subset==0').plot(x='timestamp', y=['volatile_asset_price'], color='run')

In [ ]:
use_cols = [key + '_balance' for key in cam_deposits]

In [ ]:
def get_weight_evolution(df, subset):
    df_ = pd.DataFrame(
            df.query('subset==@subset')['capital_allocation_target_weights'].to_list(),
        )
    
    df_.index = df.query('subset==@subset')['timestamp']
    return df_

In [ ]:
def get_weight_evolution_average(df, subset):
    
    balances = pd.DataFrame(df.query('subset==@subset')["capital_allocation_target_weights"].to_list(),
             columns=use_cols)
    
    df_ = df.query('subset==@subset')[['timestep']].reset_index()
    
    df_ = pd.concat([df_, balances], axis=1).set_index('timestamp')
    
    df_ = df_.groupby('timestep').mean()
    
    df_.index = df.query('subset==@subset and run==1')['timestamp']
    
    return df_

In [ ]:
get_weight_evolution_average(df, 0).plot(
    title='CAM Weight Evolution for FEI Savings Rate Setting 1, across multiple runs',
     labels={
         "value": "CAM Weights",
     },
)

In [ ]:
fig = px.area(df.query('subset==0'), x='timestamp', y=use_cols, groupnorm="percent")

fig.update_layout(
    title="FEI Capital Allocation for FEI Savings Rate Setting 1",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
fig.show()

In [ ]:
get_weight_evolution_average(df, 1).plot(
    title='CAM Weight Evolution for FEI Savings Rate Setting 2, across multiple runs',
     labels={
         "value": "CAM Weights",
     },
)

In [ ]:
fig = px.area(df.query('subset==1'), x='timestamp', y=use_cols, groupnorm="percent")

fig.update_layout(
    title="FEI Capital Allocation for FEI Savings Rate Setting 2",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
get_weight_evolution_average(df, 2).plot(
    title='CAM Weight Evolution for FEI Savings Rate Setting 3, across multiple runs',
     labels={
         "value": "CAM Weights",
     },
)

In [ ]:
fig = px.area(df.query('subset==2'), x='timestamp', y=use_cols, groupnorm="percent")

fig.update_layout(
    title="FEI Capital Allocation for FEI Savings Rate Setting 3",
    xaxis_title="Timestamp",
    yaxis_title="FEI Capital Allocation",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="right",
        x=0.99
    )
)

fig.show()

### User Circulating FEI Constituents

In [ ]:
# TODO - formatting for scientific notation
# df['your_col_here'].apply(lambda x: your_human_rounding_function(x))

In [ ]:
#df.query('subset==0')[['total_user_circulating_fei'] + use_cols]

### FEI deposit sizes

In [ ]:
def get_averages_by_subset(df, variables):
    L = []
    for i in df['subset'].value_counts().index:
        df_ = df.query('subset == @i')
        df_ = df_[['timestep', 'subset', 'run'] + variables].groupby('timestep').mean()
        df_.index = df.query('subset==@i and run==1')['timestamp']
        L.append(df_)
        
    return pd.concat(L, axis=0)

In [ ]:
fig = get_averages_by_subset(df, ['total_user_circulating_fei']).plot(
    #x='timestep',
    y='total_user_circulating_fei',
    color='subset'
)

fig.update_layout(
    title="Total User Circulating FEI",
    xaxis_title="Timestamp",
    yaxis_title="Total User Circulating FEI",
    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'total_user_circulating_fei')

In [ ]:
fig = get_averages_by_subset(df, ['fei_liquidity_pool_user_deposit_balance']).plot(
    #x='timestamp',
    y='fei_liquidity_pool_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Liquidity Pool Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI LP Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_liquidity_pool_user_deposit_balance', start=50, end=150)

In [ ]:
fig = get_averages_by_subset(df, ['fei_money_market_user_deposit_balance']).plot(
    #x='timestamp',
    y='fei_money_market_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Money Market Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI MM Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_money_market_user_deposit_balance')

In [ ]:
fig = get_averages_by_subset(df, ['fei_savings_user_deposit_balance']).plot(
    #x='timestep',
    y='fei_savings_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Savings Deposit Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI SD Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_savings_user_deposit_balance')

In [ ]:
fig = get_averages_by_subset(df, ['fei_idle_user_deposit_balance']).plot(
    #x='timestep',
    y='fei_idle_user_deposit_balance',
    color='subset',
)

fig.update_layout(
    title='User FEI Savings Idle Balance',
    xaxis_title="Timestamp",
    yaxis_title="User FEI Idle Balance",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)

fig.show()

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'fei_savings_user_deposit_balance')

### money market

In [ ]:
get_averages_by_subset(df, ['fei_money_market_borrowed']).plot(
    #x='timestamp',
    y=['fei_money_market_borrowed'],
    color='subset'
)

In [ ]:
get_averages_by_subset(df, ['fei_money_market_utilization']).plot(
    #x='timestamp',
    y=['fei_money_market_utilization'],
    color='subset'
)

In [ ]:
get_averages_by_subset(df, ['fei_money_market_supply_rate', 'fei_money_market_borrow_rate']).plot(
    #x='timestamp',
    y=['fei_money_market_supply_rate', 'fei_money_market_borrow_rate'],
    color='subset'
)

### liquidity pool

In [ ]:
get_averages_by_subset(df, ['liquidity_pool_trading_fees']).plot(
    #x='timestamp',
    y=['liquidity_pool_trading_fees'],
    color='subset'
)

In [ ]:
get_averages_by_subset(df, ['fei_liquidity_pool_user_deposit_yield_rate']).plot(
    #x='timestamp',
    y=['fei_liquidity_pool_user_deposit_yield_rate'],
    color='subset'
)

### fei savings deposit

In [ ]:
df.query('run==1').plot(x='timestamp', y=['fei_savings_rate'], color='subset')

In [ ]:
#TODO: make this sensible

In [ ]:
n_runs = len(df['subset'].value_counts())

fig = make_subplots(rows=n_runs, cols=1)

for i in range(n_runs):
    
    j = i+1
    
    df_ = df.query('subset==@i')

    fig.add_trace(
        go.Scatter(x=df_.index, y=df_['collateralization_ratio']),
        row=j, col=1
    )
    
fig.update_layout(
    title='Collateralization Ratio for each subset',
    xaxis_title="Timestamp",
    yaxis_title="Collateralization Ratio",

    legend=dict(
        yanchor="top",
        y=0.98,
        xanchor="left",
        x=0.01
    )
)
    
fig.show()

In [ ]:
# TODO: plot subplots for each of these

In [ ]:
df.query('run==1').plot(x='timestamp', y=['collateralization_ratio'], color='subset')

In [ ]:
compute_metric_set_for_variable(df, fn_dict, 'collateralization_ratio')

In [ ]:
#df.plot(x='timestamp', y=['stable_backing_ratio'], color='subset')

In [ ]:
#(df.query('subset==0')['fei_money_market_supply_rate'] - df.query('subset==0')['fei_savings_rate']).plot()

# KPIs

In [ ]:
# whats the probability that the state variable has a volatility greater than x for this policy setting?

## Conclusion

From this analyisis we can make the following qualitative conclusion - when aggregate demand for deposits is based on yield rates as the only driver of utility, deposit population will converge to the mechanism offering the highest yield. We notice specifically that in trajectory 3, given the non-instantaneous rebalance velocity, that utilization of the FSD never reaches that attained in trajectory 2, but also does not fall back below that of trajectory 1.

We furthermore notice the existance of downstream effects on money market utilization and liquidity pool yield, more greatly sustained when both these deposits have higher affluence.

We finally notice some minor downstream effects on the collateralization ratio. Here, this process is affected by changes in PCV value and PCV yield accrued, as well as changes in user circulating FEI, the former affected by money market usage and the latter by liquidity pool usage.

# Other 

In [ ]:
generate_emp_distribution_kpi(compute_vol, df, 'total_user_circulating_fei')

In [ ]:
# whats the probability that the state variable falls below x for this policy setting?

In [ ]:
generate_emp_distribution_kpi(compute_min, df, 'total_user_circulating_fei')

In [ ]:
# whats the probability that the state variable ends up at at least x for this policy setting?

In [ ]:
generate_emp_distribution_kpi(compute_final_val, df, 'total_user_circulating_fei')

In [ ]:
generate_emp_distribution_kpi(compute_mdd, df, 'total_user_circulating_fei')

In [ ]:
s = df.query('run== 1 and subset== 0')['total_user_circulating_fei']
s2 = df.query('run== 1 and subset== 2')['total_user_circulating_fei']
compute_mdd(s[150:])
compute_mdd(s2[150:])

In [ ]:
# what is the probability that CR >= 1? VA price analysis

In [ ]:
df.query('subset == 0 and run==1')['collateralization_ratio'] - df.query('subset == 1 and run==1')['collateralization_ratio']

In [ ]:
df.query('subset == 0 and run==2')['collateralization_ratio'] - df.query('subset == 1 and run==2')['collateralization_ratio']

In [ ]:
# nonparametric hypothesis test:
# statistic - difference in - collateral ratio, stable backing ratio across pairs of policies for runs
# test: h0 Delta_v2 - Delta_v1 = 0, h1: Delta_v2 - Delta_v1 =/= 0
# https://www.investopedia.com/terms/t/t-test.asp
# https://machinelearningmastery.com/nonparametric-statistical-significance-tests-in-python/